In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report 

In [2]:
df=pd.read_csv('news_dataset.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
print(df.columns,'\n\n')
print(df.info(),'\n\n')
print(df.isnull().sum())

Index(['id', 'title', 'author', 'text', 'label'], dtype='object') 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB
None 


id           0
title      558
author    1957
text        39
label        0
dtype: int64


# Data Analysis

In [5]:
df.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [6]:
df= df.dropna()


In [7]:
print(df.isna().sum(),'\n\n')
print(df.shape)


id        0
title     0
author    0
text      0
label     0
dtype: int64 


(18285, 5)


In [8]:
df.reset_index(inplace=True)
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df=df.drop(['id','text','author'],axis=1)

In [10]:
df.head()

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


In [11]:
df.columns

Index(['index', 'title', 'label'], dtype='object')

In [12]:
df

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1
...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,0
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",0
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,0
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",1


# Data Preprocessing

# 1.Tokenization

In [13]:
sample_data='The quick brown fox jumps over the lazy dog'
sample_data2=sample_data.split()
sample_data2

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [14]:
sample_data

sam_data=[data.lower() for data in sample_data]
sam_data

['t',
 'h',
 'e',
 ' ',
 'q',
 'u',
 'i',
 'c',
 'k',
 ' ',
 'b',
 'r',
 'o',
 'w',
 'n',
 ' ',
 'f',
 'o',
 'x',
 ' ',
 'j',
 'u',
 'm',
 'p',
 's',
 ' ',
 'o',
 'v',
 'e',
 'r',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'l',
 'a',
 'z',
 'y',
 ' ',
 'd',
 'o',
 'g']

# 3.Remove StopWords

In [15]:
stopwords = stopwords.words('english')
print(stopwords[0:10])
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


# 4.Stemming


In [16]:
sample_data

'The quick brown fox jumps over the lazy dog'

In [17]:
ps= PorterStemmer()
sample_data_stemming = [ps.stem(data) for data in sample_data]
print(sample_data_stemming)

['t', 'h', 'e', ' ', 'q', 'u', 'i', 'c', 'k', ' ', 'b', 'r', 'o', 'w', 'n', ' ', 'f', 'o', 'x', ' ', 'j', 'u', 'm', 'p', 's', ' ', 'o', 'v', 'e', 'r', ' ', 't', 'h', 'e', ' ', 'l', 'a', 'z', 'y', ' ', 'd', 'o', 'g']


# 5.Lemmatization

In [18]:
lm = WordNetLemmatizer()
sample_data_lemma = [lm.lemmatize(data) for data in sample_data]
print(sample_data_lemma)

['T', 'h', 'e', ' ', 'q', 'u', 'i', 'c', 'k', ' ', 'b', 'r', 'o', 'w', 'n', ' ', 'f', 'o', 'x', ' ', 'j', 'u', 'm', 'p', 's', ' ', 'o', 'v', 'e', 'r', ' ', 't', 'h', 'e', ' ', 'l', 'a', 'z', 'y', ' ', 'd', 'o', 'g']


In [19]:
lm =WordNetLemmatizer()
corpus =[]
for i in range(len(df)):
    review =re.sub('^a-zA-Z0-9',' ',df['title'][i])
    review=review.lower()
    review = review.split()
    review = [lm.lemmatize(x)for x in review if x not in stopwords]
    review=" ".join(review)
    corpus.append(review)

In [20]:
len(corpus)

18285

In [21]:
df['title'][4]

'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [22]:
corpus[4]

'iranian woman jailed fictional unpublished story woman stoned death adultery'

# Vectorization

In [23]:
tf=TfidfVectorizer()
x=tf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
y=df.label
y

0        1
1        0
2        1
3        1
4        1
        ..
18280    0
18281    0
18282    0
18283    1
18284    1
Name: label, Length: 18285, dtype: int64

# Data Spliting

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=10,stratify =y)

In [26]:
print(len(x_train),len(y_train),len(x_test),len(y_test))

12799 12799 5486 5486


# Model Building

In [27]:
rf= RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

# Model Evaluation

In [28]:
y_pred = rf.predict(x_test)
accuracy_score_ = accuracy_score(y_test,y_pred) 
accuracy_score_

0.9363835216915786

In [37]:
class Evaluation:
    def __init__(self,model,x_train,x_test,y_train,y_test):
        self.model =model
        self.x_train = x_train
        self.x_test=x_test
        self.y_train = y_train
        self.y_test=y_test
        
    def train_evaluation(self):
        y_pred_train =self.model.predict(self.x_train)
        
        acc_scr_train =accuracy_score(self.y_train,y_pred_train)
        print("Accuracy Score of Training set :\n",acc_scr_train)
        
        con_mat_train =confusion_matrix(self.y_train,y_pred_train)
        print("Confusion Matrix on Training dataset :\n ",con_mat_train)
        
        class_rep_train=classification_report(self.y_train,y_pred_train)
        print("Classifiaction Report of Training Set :\n",class_rep_train)
        
    def test_evaluation(self):
        y_pred_test =self.model.predict(self.x_test)
        
        acc_scr_test =accuracy_score(self.y_test,y_pred_test)
        print("Accuracy Score of Training set :\n",acc_scr_test)
        
        con_mat_test =confusion_matrix(self.y_test,y_pred_test)
        print("Confusion Matrix on Training dataset :\n ",con_mat_test)
        
        class_rep_test=classification_report(self.y_test,y_pred_test)
        print("Classifiaction Report of Training Set :",class_rep_test)

In [38]:
print(Evaluation(rf,x_train,x_test,y_train,y_test).train_evaluation())
print('\n\n')

print(Evaluation(rf,x_train,x_test,y_train,y_test).test_evaluation())

Accuracy Score of Training set :
 1.0
Confusion Matrix on Training dataset :
  [[7252    0]
 [   0 5547]]
Classifiaction Report of Training Set :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7252
           1       1.00      1.00      1.00      5547

    accuracy                           1.00     12799
   macro avg       1.00      1.00      1.00     12799
weighted avg       1.00      1.00      1.00     12799

None



Accuracy Score of Training set :
 0.9363835216915786
Confusion Matrix on Training dataset :
  [[2817  292]
 [  57 2320]]
Classifiaction Report of Training Set :               precision    recall  f1-score   support

           0       0.98      0.91      0.94      3109
           1       0.89      0.98      0.93      2377

    accuracy                           0.94      5486
   macro avg       0.93      0.94      0.94      5486
weighted avg       0.94      0.94      0.94      5486

None


# Prediction Pipeline

In [43]:
class Preprocessing:
    def __init__(self,data):
        self.data=data
    def text_preprocessing(self):
        lm=WordNetLemmatizer()
        pred_data=[self.data]
        preprocess_data=[]
        
        for data in pred_data:
            review= re.sub('^a-zA-Z0-9',' ',data)
            review= review.lower()
            review=review.split()
            review=[lm.lemmatize(x) for x in review if x not in stopwords]
            review=" ".join(review)
            preprocess_data.append(review)
        return preprocess_data

In [44]:
data = df.title[4]
data

'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [45]:
Preprocessing(data).text_preprocessing()

['iranian woman jailed fictional unpublished story woman stoned death adultery']

In [46]:
class Prediction:
    def __init__(self,pred_data,model):
        self.pred_data =pred_data
        self.model=model
        
    def prediction_model(self):
        preprocess_data=Preprocessing(self.pred_data).text_preprocessing()
        data =tf.transform(preprocess_data)
        prediction =self.model.predict(data)
        
        if(prediction[0] ==0):
            return "News is Fake"
        else:
            return "News is Real"

In [47]:
data

'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [49]:
Prediction(data,rf).prediction_model()

'News is Real'

In [50]:
da ="Hi I am an unlucky girl"
Prediction(da,rf).prediction_model()

'News is Real'

In [51]:
d_a = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
Prediction(d_a,rf).prediction_model()

'News is Fake'